# Assignment 6: scikit-learn
- **you will learn:** How to build end-to-end preprocessing pipelines, combine them with different regression models, and evaluate their performance
- **task:**  See section 6.6 below
- **deadline:** 15.12.2025
- [scikit-learn webpage](https://scikit-learn.org/)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.
---

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit learn version:", sk.__version__)

## **6.1 Introduction to scikit-learn**

In this session, we will learn how to use **scikit-learn** not only as a library of machine learning models,
but mainly as a **framework** for:

- data preparation and preprocessing,
- building end-to-end pipelines,
- reproducible and correct model evaluation.

We intentionally avoid complex models.
Our goal is to understand the **scikit-learn workflow**, not to achieve state-of-the-art performance.

###  *What is scikit-learn?*

Scikit-learn is built on the concept of **estimators** — objects that follow a simple and unified API:

- `fit(X, y)` — learn from the data
- `predict(X)` — make predictions
- `transform(X)` — transform the data (for preprocessing)

Everything in scikit-learn that learns from data is an **estimator**.

Before diving deeper, here is the simplest possible sklearn model:

```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)
```

### *Estimators vs Transformers*

- **Estimator**: any object that implements `fit()`.
  It can **learn from data**.
  Examples: LinearRegression, RandomForestRegressor, StandardScaler.

- **Transformer**: a special type of estimator that also implements `transform()`.
  It can **modify data** (e.g., scale, encode, impute).
  Examples: StandardScaler, OneHotEncoder, SimpleImputer.

- **Model vs Transformer**:
  - Model: usually `fit` + `predict`
  - Transformer: usually `fit` + `transform` (or `fit_transform`)
  - Pipelines can combine both into a single workflow.

> In short: *Every transformer is an estimator, but not every estimator is a transformer.*

### *The typical scikit-learn workflow*

1. Load the dataset
2. Inspect the structure of the data
3. Split the dataset into train/test
4. Create preprocessing steps
5. Combine preprocessing + model into a **Pipeline**
6. Train (`fit`)
7. Validate on independent data

---

### *Dataset*

We use the built-in **California Housing** dataset.
The task is to predict the median house value in different regions of California.

In [ ]:
from sklearn.datasets import fetch_california_housing

# Load dataset
X_old, y = fetch_california_housing(return_X_y=True, as_frame=True)
X = X_old.copy()

# Create one categorical feature based on HouseAge
X['HouseAge_cat'] = pd.cut(
    X['HouseAge'],
    bins=[0, 20, 50, np.inf],
    labels=['new', 'medium', 'old']
)

# Basic checks
print("X shape:", X.shape)
print("y size:", y.size)
print("Column types:\n", X.dtypes)

print("Head:")
display(X.head())

print("Description:")
display(X.describe())


## 6.2 **Train-test split**

In machine learning, we usually split our dataset into:

- **Training set** – used to fit the model
- **Test set** – used to evaluate the model on unseen data

This is important to check if our model **generalizes** to new data.
Without a test set, we might think our model is perfect, but it could just be **overfitting** to the training data.



We will use `sklearn.model_selection.train_test_split` for this.


In [ ]:
from sklearn.model_selection import train_test_split
# Simple 80/20 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,        # 20% of data for testing
    random_state=42,      # reproducibility
    shuffle=True          # shuffle data before splitting
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

### Key Points

- Always split the data **before any preprocessing or model fitting**.
- `test_size=0.2` is common, but can be adjusted depending on dataset size.
- `random_state` ensures reproducibility: every time you use the same seed, the **random shuffling** happens the same way.
- `shuffle=True` (default) ensures training and test sets are representative.
  Use `shuffle=False` for time-dependent data.
- Check that the target distribution in train and test sets is roughly similar:

In [ ]:
import matplotlib.pyplot as plt

# Histogram of target for training and test set
plt.figure(figsize=(12,5))

# Training set
plt.subplot(1, 2, 1)
plt.hist(y_train, bins=30, color='skyblue', edgecolor='black')
plt.title("Training set target distribution")
plt.xlabel("Median House Value")
plt.ylabel("Count")

# Test set
plt.subplot(1, 2, 2)
plt.hist(y_test, bins=30, color='salmon', edgecolor='black')
plt.title("Test set target distribution")
plt.xlabel("Median House Value")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

## **6.3 Preprocessing**

Before feeding data into a model, we often need to **preprocess** it.
Typical preprocessing steps:

- Scale numerical features
- Encode categorical features  

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

### What is a Transformer?

A **transformer** is an object that:

- Implements `fit()` – learns from the data
- Implements `transform()` – modifies data (e.g., scaling, encoding)
- Often provides `fit_transform()` as a shortcut

Examples:

- `StandardScaler` – scales numerical features
- `OneHotEncoder` – converts categorical variables into numeric format


### OneHotEncoder example

We take a small categorical column and see how OneHotEncoder works:


In [ ]:
# Example categorical column
df = pd.DataFrame({"Color": ["red", "green", "blue", "green"]})

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded = encoder.fit_transform(df)

print("Original column:\n", df)
print("Feature names:", encoder.get_feature_names_out())
print("Encoded array:\n", encoded)

### Combining transformers with ColumnTransformer

Suppose we have numerical and categorical columns:

- **Numerical**: continuous numbers
- **Categorical**: discrete categories

We can apply different transformers to different columns using `ColumnTransformer`, which allows you to **combine multiple transformers in one object**, which can then be directly used in a pipeline with a model.:

In [ ]:
# Select features
numeric_features = ["MedInc", "Population"]
categorical_features = ["HouseAge_cat"]

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')

# Combine transformers in a ColumnTransformer
# ---------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"  # other columns will be dropped
)

# Fit and transform training data ONLY
X_train_processed = preprocessor.fit_transform(X_train)
print("Processed training feature matrix shape:", X_train_processed.shape)
print("Processed training feature matrix:\n", X_train_processed[:5]) # show first 5 rows


# Compute and print mean of numeric features in training data
train_numeric_means = np.mean(X_train_processed[:, :len(numeric_features)], axis=0)
print("Mean of numeric features in training data (should be ~0):")
for name, mean_val in zip(numeric_features, train_numeric_means):
    print(f"{name}: {mean_val:.4f}")

In [ ]:
# Transform test data using the preprocessor fitted on training data
X_test_processed = preprocessor.transform(X_test)
print("Processed test feature matrix shape:", X_test_processed.shape)
print("Test data transformed using the same parameters as training data:")
print(X_test_processed[:5])  # show first 5 rows


# Compute and print mean of numeric features in test data
test_numeric_means = np.mean(X_test_processed[:, :len(numeric_features)], axis=0)
print("Mean of numeric features in test data (may NOT be 0):")
for name, mean_val in zip(numeric_features, test_numeric_means):
    print(f"{name}: {mean_val:.4f}")


### Other Common Transformers in scikit-learn

Scikit-learn provides many transformers to preprocess or modify data. Some common ones:

  - `MinMaxScaler()` → scales features to a fixed range [0,1]
  - `RobustScaler()` → scales features using median and IQR (robust to outliers)
  - `OrdinalEncoder()` → converts categories to integer labels
  - `PolynomialFeatures(degree=2)` → generates polynomial and interaction features
  - `SimpleImputer(strategy="mean")` → fills missing values with mean
  - `SimpleImputer(strategy="most_frequent")` → fills missing with mode


## **6.4 Pipeline: Combining Preprocessing with a Model**

In scikit-learn, the recommended workflow is to combine all preprocessing steps
(imputation, scaling, encoding, etc.) with the machine learning model itself
into a single unified `Pipeline`.

### Why use a pipeline?

- The model receives **clean, consistently preprocessed data** every time.
- Preprocessing is automatically applied during:
  - fitting on the training set,
  - evaluation on the test/validation set,
  - later inference on new unseen data.
- The workflow becomes **clean, modular, reproducible, and maintainable**.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Attach the model to the preprocessing pipeline
model = Pipeline(steps=[
    ("preprocessing", preprocessor),  # previously defined ColumnTransformer
    ("regressor", LinearRegression())  # any scikit-learn estimator
])

- The first element in each step of the pipeline is a **name** (string) used for reference.
- The second element is a **transformer or estimator** object.
- The last step must be an **estimator** (e.g., `LinearRegression`, `RandomForestRegressor`, etc.).
- All intermediate steps must implement `fit` and `transform`. The final step must implement `fit` and `predict`.
- Using a `Pipeline` ensures that the **entire workflow** (preprocessing + model) is evaluated consistently.
  - Example: during cross-validation, preprocessing is applied only on training folds.
- Prevents **data leakage** by ensuring that preprocessing statistics (e.g., mean/std) are learned **only from training data**.
- Makes the workflow **modular, reproducible, and maintainable**.

## **6.5 Fitting and Evaluating on the Test Set**

- Now that we have a complete `Pipeline` (preprocessing + model), we can **fit it** on the training data:
  - All preprocessing steps are applied automatically.
  - The model is trained on the transformed features.
- This ensures that the **entire workflow** is applied consistently and prevents data leakage.


In [ ]:
model.fit(X_train, y_train)

In [ ]:
# The last step in the pipeline is the regressor
regressor = model.named_steps["regressor"]

# Coefficients of the linear regression model
coefficients = regressor.coef_
intercept = regressor.intercept_

print("Intercept:", intercept)
print("Coefficients:", coefficients)

# Optionally, match coefficients to feature names
# For numeric features + one-hot encoded categorical features, we need names
# Get numeric feature names
numeric_features = ["MedInc", "Population"]
# Get categorical feature names after one-hot encoding
cat_feature_names = model.named_steps["preprocessing"].named_transformers_["cat"].get_feature_names_out(["HouseAge_cat"])

# Combine all feature names
feature_names = numeric_features + list(cat_feature_names)

# Print coefficients with feature names
for name, coef in zip(feature_names, coefficients):
    print(f"{name}: {coef:.4f}")


Once we have trained our full pipeline (preprocessing + model), the last step is
to evaluate how well it performs on data that the model has **never seen** — the
test set.


### Why evaluate on a separate test set?

- To simulate real-world performance on new unseen data.
- To detect overfitting (model performs much better on training data than on test data).
- To estimate the generalization ability of our pipeline.

### Common regression metrics

- **R² score**: measures proportion of variance explained by the model.
  - R² = 1 → perfect predictions
  - R² = 0 → model is no better than predicting the mean
- **Mean Absolute Error (MAE)**: average absolute difference between predictions and true values.
- **Root Mean Squared Error (RMSE)**: square root of MSE; penalizes larger errors more and is in the same units as the target.

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Predict on the test data
y_pred = model.predict(X_test)

# Compute evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Display results
print(f"Test R² score:       {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"RMSE:                {rmse:.4f}")

## 6.6  **Homework: Custom Preprocessing + Model Comparison**

Your goal is to design a custom preprocessing pipeline for the California Housing dataset and compare the performance of three different regression models.
Because several transformations must be applied to the **same columns in sequence**, you must use **Pipeline** objects inside a **ColumnTransformer**.


### 1. Dataset (same as before)
Use the California Housing dataset. Split the data into `train` and `test`.

### 2. Preprocessing Requirements

#### A. Numerical features using **StandardScaler → PolynomialFeatures**
Apply these two transformations **in this exact order** to the features:

- `MedInc`
- `Population`

Steps:
1. First apply `StandardScaler()`
2. Then apply `PolynomialFeatures(degree=2, include_bias=False)`

#### B. Numerical features using **MinMaxScaler → PolynomialFeatures**
Apply these two transformations **in this exact order** to the features:

- `Latitude`
- `Longitude`

Steps:
1. First apply `MinMaxScaler(feature_range=(0, 1))`
2. Then apply `PolynomialFeatures(degree=2, include_bias=False)`

#### C. Categorical features using OneHotEncoder (same as before)
Apply:

- `OneHotEncoder(handle_unknown="ignore")`

to the feature:

- `HouseAge_cat`

### 3. How to chain multiple transformations
Because you need two transformations applied sequentially to the same variables, you must wrap them inside `Pipeline([...])`.

Example structure (you must implement both variants):

```
Pipeline([
    ("scaler", Transformer_1),
    ("poly", Transformer_2)
])
```

Then include these pipelines inside a `ColumnTransformer`.

## 4. Combine everything
Use a `ColumnTransformer` with three parts:

1. StandardScaler → PolynomialFeatures for (`MedInc`, `Population`)
2. MinMaxScaler → PolynomialFeatures for (`Latitude`, `Longitude`)
3. OneHotEncoder for (`HouseAge_cat`)

## 5. Models to train
Train the following three models on the **same preprocessing pipeline**:

1. `LinearRegression`
2. `Ridge` (with default parameters)
3. `DecisionTreeRegressor` (with default parameters)

For each model, create a full pipeline:

```
Pipeline([
    ("preprocess", preprocessor),
    ("regressor", ...)
])
```
Fit all three models on the training set.


## 6. Evaluation
For each model, compute the **RMSE on the test set**:

```
RMSE = sqrt(mean_squared_error(y_test, y_pred))
```

Print the test RMSE for all three models and compare their performance.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor